# Ex2

## Q1

TBD

## Q2

TBD

## Q3

TBD

## Q4

### Get Corpus and Dictionary

In [1]:
import urllib2
import zipfile
import os

data_folder = 'corpus_combined'
corpus_file = data_folder + '.zip'

if not os.path.exists(data_folder):
    os.makedirs(data_folder)
    
zipcontent = urllib2.urlopen('http://www2.mta.ac.il/~gideon/courses/nlp/data/' + corpus_file).read()
with open(corpus_file, 'w') as f:
    f.write(zipcontent)
zip_ref = zipfile.ZipFile(corpus_file, 'r')
zip_ref.extractall(data_folder)
zip_ref.close()
os.remove(corpus_file)

response = urllib2.urlopen('http://www2.mta.ac.il/~gideon/courses/nlp/data/word_list_20k.txt')
dictionary = response.read().split('\n')
dictionary.remove('')

### Prepare training and test set

In [2]:
import urllib2

# training
kidnapped = urllib2.urlopen('http://www.gutenberg.org/files/421/421-0.txt').read().decode('utf8')
oliver_twist = urllib2.urlopen('http://www.gutenberg.org/ebooks/730.txt.utf-8').read().decode('utf8')
training_set = [kidnapped, oliver_twist]

# testing
persuasion = open(data_folder + '/persuasion.txt', 'r').read()
testing_set = [persuasion]

### Hyperparameter Tuning

In [3]:
hyperparameters = {
    'lowercase': True,
    'stemming': False,
    'remove_stopwords': False,
    'max_length': -1
}

### Pre-Processing

In [4]:
from chapterize import Book
from nltk.tokenize import word_tokenize

OOV = 'OOV'
NUM = 'NUM'

# removing header, footer and chapters
training = ''
for book in training_set:
    training += Book(book).getContent()
testing = ''
for book in testing_set:
    testing += Book(book).getContent() + ' '

# lowercasing
if hyperparameters['lowercase']:
    training = training.lower()
    testing = testing.lower()
    dictionary = [word.lower() for word in dictionary]
    
# tokenizing
training = word_tokenize(training)
testing = word_tokenize(testing)

# removing stopwords
if hyperparameters['remove_stopwords']:
    from nltk.corpus import stopwords
    training = [token for token in training if token not in set(stopwords.words('english'))]
    testing = [token for token in testing if token not in set(stopwords.words('english'))]
    dictionary = [word for word in dictionary if word not in set(stopwords.words('english'))]
    
# stemming
if hyperparameters['stemming']:
    from nltk.stem import PorterStemmer
    ps = PorterStemmer()
    training = [ps.stem(token) for token in training]
    testing = [ps.stem(token) for token in testing]
    dictionary = [ps.stem(word) for word in dictionary]

dictionary.append(OOV)
dictionary.append(NUM)

# replacing OOV and NUM tokens
missing_tokens = set([token for token in training if token not in dictionary])
for (i, token) in enumerate(training):
    if token in missing_tokens:
        if token.isdigit():
            training[i] = NUM
        else:
            training[i] = OOV

missing_tokens = set([token for token in testing if token not in dictionary])
for (i, token) in enumerate(testing):
    if token in missing_tokens:
        if token.isdigit():
            testing[i] = NUM
        else:
            testing[i] = OOV

### Unigram

#### Training

In [5]:
unigram_model = {word: training.count(word) for word in dictionary}

#### Testing

In [6]:
from __future__ import division

unigram_model['OOV'] / len(training)

0.22163537649511159

### Bigram

#### Training

In [9]:
from nltk import bigrams
from itertools import product

bigram_model = {}
bigram_tokens = list(bigrams(training))
for token in bigram_tokens:
    bigram_model[token] = bigram_model.get(token, 0) + 1

#### Testing

In [10]:
from __future__ import division

bigram_model.get(('they', 'are'), 0) / len(bigram_tokens)

9.272041703583129e-05

### Trigram

In [11]:
from nltk import trigrams
from itertools import product

trigram_model = {}
trigram_tokens = list(trigrams(training))
for token in trigram_tokens:
    trigram_model[token] = trigram_model.get(token, 0) + 1

In [12]:
from __future__ import division

trigram_model.get(('here', 'it', 'is'), 0) / len(trigram_tokens)

3.4341013128569318e-06